# Setup

In [2]:
!pip install findspark
!pip install pyarrow==0.14

In [3]:
import findspark
findspark.init()

In [1]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20211202181731-0000
KERNEL_ID = 6ce2889c-5487-4b9a-88df-ecc8fd24a467


In [2]:
!python --version

Python 3.7.10


In [3]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.0.2
      /_/
                        
Using Scala version 2.12.10, Eclipse OpenJ9 VM, 1.8.0_252
Branch HEAD
Compiled by user centos on 2021-02-16T04:53:13Z
Revision 648457905c4ea7d00e3d88048c63f360045f0714
Url https://gitbox.apache.org/repos/asf/spark.git
Type --help for more information.


# Spark Context and Spark Session

### Creating the spark session and context

In [5]:
sc = SparkContext()

spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Initialize spark session

In [6]:
spark

# Load the data and creating a table view

### Loading data into pandas DataFrame

In [7]:
mtcars = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/mtcars.csv")

In [8]:
mtcars.head()

,Unnamed: 0,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [9]:
mtcars.rename({"Unnamed: 0": "name"}, inplace=True)

### Loading data into the Spark DataFrame

In [10]:
sdf = spark.createDataFrame(mtcars)

In [11]:
sdf.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- mpg: double (nullable = true)
 |-- cyl: long (nullable = true)
 |-- disp: double (nullable = true)
 |-- hp: long (nullable = true)
 |-- drat: double (nullable = true)
 |-- wt: double (nullable = true)
 |-- qsec: double (nullable = true)
 |-- vs: long (nullable = true)
 |-- am: long (nullable = true)
 |-- gear: long (nullable = true)
 |-- carb: long (nullable = true)



### Create a table view

In [12]:
sdf.createTempView("cars")

# Running SQL queries and aggregating data

In [13]:
# showing all the data
spark.sql("SELECT * FROM cars").show()

+-------------------+----+---+-----+---+----+------------------+-----+---+---+----+----+
|         Unnamed: 0| mpg|cyl| disp| hp|drat|                wt| qsec| vs| am|gear|carb|
+-------------------+----+---+-----+---+----+------------------+-----+---+---+----+----+
|          Mazda RX4|21.0|  6|160.0|110| 3.9|              2.62|16.46|  0|  1|   4|   4|
|      Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|             2.875|17.02|  0|  1|   4|   4|
|         Datsun 710|22.8|  4|108.0| 93|3.85|              2.32|18.61|  1|  1|   4|   1|
|     Hornet 4 Drive|21.4|  6|258.0|110|3.08|             3.215|19.44|  1|  0|   3|   1|
|  Hornet Sportabout|18.7|  8|360.0|175|3.15|              3.44|17.02|  0|  0|   3|   2|
|            Valiant|18.1|  6|225.0|105|2.76|              3.46|20.22|  1|  0|   3|   1|
|         Duster 360|14.3|  8|360.0|245|3.21|              3.57|15.84|  0|  0|   3|   4|
|          Merc 240D|24.4|  4|146.7| 62|3.69|              3.19| 20.0|  1|  0|   4|   2|
|           Merc 230|

In [15]:
# showing a specific column
spark.sql("SELECT mpg FROM cars").show(5)

+----+
| mpg|
+----+
|21.0|
|21.0|
|22.8|
|21.4|
|18.7|
+----+
only showing top 5 rows



In [16]:
# Basic filtering operation
spark.sql("SELECT * FROM cars WHERE mpg>20 AND cyl<6").show(5)

+-----------+----+---+-----+---+----+-----+-----+---+---+----+----+
| Unnamed: 0| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------+----+---+-----+---+----+-----+-----+---+---+----+----+
| Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|  Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|
|   Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|
|   Fiat 128|32.4|  4| 78.7| 66|4.08|  2.2|19.47|  1|  1|   4|   1|
|Honda Civic|30.4|  4| 75.7| 52|4.93|1.615|18.52|  1|  1|   4|   2|
+-----------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



In [18]:
# Aggregating data
spark.sql("SELECT cyl, count(*) as count FROM cars GROUP BY cyl ORDER BY cyl").show(5)

+---+-----+
|cyl|count|
+---+-----+
|  4|   11|
|  6|    7|
|  8|   14|
+---+-----+



# Create a Pandas UDF to apply a columnar operation

Apache Spark has become the de-facto standard in processing big data. To enable data scientists to leverage the value of big data, Spark added a Python API in version 0.7, with support for user-defined functions (UDF). These user-defined functions operate one-row-at-a-time, and thus suffer from high serialization and invocation overhead. As a result, many data pipelines define UDFs in Java and Scala and then invoke them from Python.

Pandas UDFs built on top of Apache Arrow bring you the best of both worlds—the ability to define low-overhead, high-performance UDFs entirely in Python. In this simple example, we will build a Scalar Pandas UDF to convert the wT column from imperial units (1000-lbs) to metric units (metric tons).

In addition, UDFs can be registered and invoked in SQL out of the box by registering a regular python function using the @pandas_udf() decorator.

### Importing libraries and registering a UDF

In [20]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [21]:
@pandas_udf("float")
def convert_wt(s: pd.Series) -> pd.Series:
    return s * 0.45

spark.udf.register("convert_weight", convert_wt)

<function __main__.convert_wt>

### Applying the UDF to the tableview

In [22]:
spark.sql("SELECT *, wt AS weight_imperial, convert_weight(wt) as weight_metric FROM cars").show()

+-------------------+----+---+-----+---+----+------------------+-----+---+---+----+----+------------------+-------------+
|         Unnamed: 0| mpg|cyl| disp| hp|drat|                wt| qsec| vs| am|gear|carb|   weight_imperial|weight_metric|
+-------------------+----+---+-----+---+----+------------------+-----+---+---+----+----+------------------+-------------+
|          Mazda RX4|21.0|  6|160.0|110| 3.9|              2.62|16.46|  0|  1|   4|   4|              2.62|        1.179|
|      Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|             2.875|17.02|  0|  1|   4|   4|             2.875|      1.29375|
|         Datsun 710|22.8|  4|108.0| 93|3.85|              2.32|18.61|  1|  1|   4|   1|              2.32|        1.044|
|     Hornet 4 Drive|21.4|  6|258.0|110|3.08|             3.215|19.44|  1|  0|   3|   1|             3.215|      1.44675|
|  Hornet Sportabout|18.7|  8|360.0|175|3.15|              3.44|17.02|  0|  0|   3|   2|              3.44|        1.548|
|            Valiant|18.

# End spark session

In [23]:
spark.stop()